In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn import Conv2d
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.transforms import CenterCrop
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
import numpy as np

In [4]:
import os
from torch.utils.data import random_split

In [5]:
# SAANVI COMPUTER CODE
data = '/content/drive/MyDrive/TrainingData_Final'
print()
dirs = os.listdir(data)
dataset_old = []
for dir in dirs:
  for inner_dir in os.listdir(os.path.join(data, dir)):
      dataset_old.append(os.path.join(data, dir, inner_dir))
# print(dataset_old)
dataset = []
curr = 0
for i in dataset_old:
  if i[42] != '5':
    dataset.append(i)
  elif curr < 400:
    dataset.append(i)
    curr += 1
#42nd index
# print(len(dataset), curr)

In [6]:
# SAANVI COMPUTER CODE
train_set = []
test_set = []
validation_set = []
for i in range(0, 6):
  path = f"/content/drive/MyDrive/TrainingData_Final/{i}"
  all_imgs_0 = os.listdir(path)
  # /content/drive/MyDrive/TrainingData_Final/0
  all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
  # print(all_imgs_0)
  if i == 5:
    all_imgs_0 = all_imgs_0[:400]
    # print(len(all_imgs_0))
  num_imgs_0 = len(all_imgs_0)
  arr = np.arange(num_imgs_0)
  np.random.shuffle(arr)
  # print(i, 'total:', num_imgs_0)
  curr_tr = len(train_set)
  curr_te = len(test_set)
  curr_va = len(validation_set)
  train_set += [all_imgs_0[idx] for idx in arr[0:int(0.8*num_imgs_0)]]
  test_set += [all_imgs_0[idx]  for idx in arr[int(0.8*num_imgs_0):int(0.9*num_imgs_0)]]
  validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
  print(i, 'train: ', len(train_set) - curr_tr)
  print(i, 'test: ', len(test_set) - curr_te)
  print(i, 'val: ', len(validation_set) - curr_va)
  print('---')
print('total:', len(train_set) + len(test_set) + len(validation_set))
print('train:', len(train_set))
print('test:', len(test_set))
print('val:', len(validation_set))

0 train:  439
0 test:  55
0 val:  55
---
1 train:  796
1 test:  100
1 val:  100
---
2 train:  128
2 test:  16
2 val:  17
---
3 train:  153
3 test:  19
3 val:  20
---
4 train:  28
4 test:  3
4 val:  4
---
5 train:  320
5 test:  40
5 val:  40
---
total: 2333
train: 1864
test: 233
val: 236


In [7]:
# SAANVI COMPUTER CODE
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

labels = []
class CustomDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize the image to (256, 256)
            transforms.ToTensor(),          # Convert the PIL image to a PyTorch tensor
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)


        return image, file_path[42]
train_dataloader = DataLoader(CustomDataset(train_set), batch_size=64, shuffle=True)
test_dataloader = DataLoader(CustomDataset(test_set), batch_size=64, shuffle=True)
val_dataloader = DataLoader(CustomDataset(validation_set), batch_size=64, shuffle=True)

In [8]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [9]:
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0].squeeze()
label = train_labels[0]

In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 5), #nn.Linear(10000, 6)
      nn.Softmax(dim=1)
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [21]:
class NeuralNetwork2(nn.Module):
  def __init__(self):
    super(NeuralNetwork2, self).__init__()
    self.linear_relu_stack = nn.Sequential(
      nn.Conv2d(3, 4, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(4, 8, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Conv2d(8, 16, kernel_size=3),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride = 2),
      nn.Flatten(),
      nn.Linear(14400, 5000),
      nn.Linear(5000, 2), #nn.Linear(10000, 6)
      # nn.Sigmoid(),
      # nn.softmax
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [22]:
def label_changer(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 3 or y[i] == 5:
      y[i] = 0
    elif y[i] != 4:
      y[i] += 1
    num = y[i]
    y[i] = torch.tensor([[0, 0, 0, 0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  return y
from re import I
# label 0, 1, 2, 4 (Diabetic retinopathy)
# og labels
#     0: "Mild Diabetic Nonproliferative Retinopathy",
#     1: "Moderate Diabetic Nonproliferative Retinopathy",
#     2: "Severe Diabetic Nonproliferative Retinopathy",
#     3: "Hypertensive Retinopathy",
#     4: "Proliferative Diabetic Retinopathy",
#     5: "No Retinopathy"
# new labels
#     0: 3, 5
#     1: 0
#     2: 1
#     3: 2
#     4: 4
model = NeuralNetwork().to(device)
print(model)
# model.summary()
batch_size = 64
# epochs = 5
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
  # Compute prediction and loss
    X = X.cpu()
    #y.expand(-1, 4)
    y = list(y)
    y = label_changer(y)
    y = y.cpu()
    pred = model(X)
    pred = pred.to(torch.float32)
    y = y.to(torch.float32)
    # print(y.shape)
    loss = loss_fn(pred, y.argmax(1))
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    # print(y)
    # print(pred)
    # print(pred.argmax(1))
    # print('pred', pred)
    # print('y', y)
    accuracy = (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    print(accuracy)
    # print('---')
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X = X.cpu()
      y = list(y)
      y = label_changer(y)
      y = y.cpu()
      pred = model(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 20
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=14400, out_features=5000, bias=True)
    (11): Linear(in_features=5000, out_features=5, bias=True)
    (12): Softmax(dim=1)
  )
)
Epoch 1
-------------------------------
loss: 1.610418 [    0/ 1864]
4.0
loss: 1.389208 [   64/ 1864]
33.0
loss: 1.529833 [  128/ 1864]
24.0
loss: 1.592333 [  192/ 1864]
20.0
loss: 1.451708 [  256/ 1864]
29.0
loss: 1.467333 [  320/ 1864]
28.0
loss: 1.576708 [  384/ 1864]
21.0


KeyboardInterrupt: ignored

In [23]:
# label 0: Mild Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 0:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
      print(pred.argmax(1))
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.690829 [    0/ 1864]
loss: 4.086617 [   64/ 1864]
loss: 2.495634 [  128/ 1864]
loss: 0.618161 [  192/ 1864]
loss: 5.712082 [  256/ 1864]
loss: 1.707914 [  320/ 1864]
loss: 2.083668 [  384/ 1864]
loss: 1.572575 [  448/ 1864]
loss: 1.180959 [  512/ 1864]
loss: 1.066002 [  576/ 1864]
loss: 0.571843 [  640/ 1864]
loss: 1.601277 [  704/ 1864]
loss: 1.191836 [  768/ 1864]
loss: 0.561246 [  832/ 1864]
loss: 0.511528 [  896/ 1864]
loss: 1.198717 [  960/ 1864]
loss: 1.093388 [ 1024/ 1864]
loss: 1.197170 [ 1088/ 1864]
loss: 0.806767 [ 1152/ 1864]
loss: 0.485874 [ 1216/ 1864]
loss: 0.503896 [ 1280/ 1864]
loss: 0.658843 [ 1344/ 1864]
loss: 0.725529 [ 1408/ 1864]
loss: 0.745004 [ 1472/ 1864]
loss: 0.675302 [ 1536/ 1864]
loss: 0.543743 [ 1600/ 1864]
loss: 0.568277 [ 1664/ 1864]
loss: 0.642776 [ 1728/ 1864]
loss: 0.484085 [ 1792/ 1864]
loss: 0.922915 [  232/ 1864]
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

In [ ]:
torch.save(model2.state_dict(),  "binary0model")

In [ ]:
# label 1: Moderate Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 1:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary1model")

In [ ]:
# label 2: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 2:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary2model")

In [ ]:
# label 3: Hypertensive  Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 3:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary3model")

In [ ]:
# label 4: Severe Diabetic Nonproliferative Retinopathy

def label_changer2(y):
  for i in range(len(y)):
    y[i] = int(y[i])
    if y[i] == 4:
      y[i] = 1
    else:
      y[i] = 0
    num = y[i]
    y[i] = torch.tensor([[0, 0]], dtype=torch.float32)
    y[i][0][num] = float(1)
  y = torch.cat(y, dim=0)
  # print(y)
  return y
model2 = NeuralNetwork2().to(device)
batch_size = 64
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
def train_loop(dataloader, model2, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.cpu()
    y = list(y)
    y = label_changer2(y)
    y = y.cpu()
    pred = model2(X)
    loss = loss_fn(pred, y.argmax(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model2, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.cpu()
      y = list(y)
      y = label_changer2(y)
      y = y.cpu()
      pred = model2(X)
      test_loss += loss_fn(pred, y.argmax(1)).item()
      correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 7
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model2, loss_fn, optimizer)
  test_loop(test_dataloader, model2, loss_fn)
print("Done!")

In [ ]:
torch.save(model2.state_dict(),  "binary4model")

## Collapsed Commented Out

In [ ]:
# # label 1
# def train_loop(dataloader, model, loss_fn, optimizer):
#   size = len(dataloader.dataset)
#   for batch, (X, y) in enumerate(dataloader):
#   # Compute prediction and loss
#     X = X.cpu()
#     y = list(y)
#     for i in range(len(y)):
#       y[i] = int(y[i])
#       if y[i] == 1:
#         y[i] = 1
#       else:
#         y[i] = 0
#     y = torch.tensor(y)
#     y = y.cpu()
#     pred = model(X)
#     loss = loss_fn(pred, y)
#     # Backpropagation
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if batch % 100 == 0:
#       loss, current = loss.item(), batch * len(X)
#       print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
# def test_loop(dataloader, model, loss_fn):
#   size = len(dataloader.dataset)
#   test_loss, correct = 0, 0
#   with torch.no_grad():
#     for X, y in dataloader:
#       X = X.cpu()
#       y = list(y)
#       for i in range(len(y)):
#         y[i] = int(y[i])
#         if y[i] == 1:
#           y[i] = 1
#         else:
#           y[i] = 0
#       y = torch.tensor(y)
#       y = y.cpu()
#       pred = model(X)
#       test_loss += loss_fn(pred, y).item()
#       correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= size # (TP + TN)/(TP + TN + FP + FN) i think
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# epochs = 7
# for t in range(epochs):
#   print(f"Epoch {t+1}\n-------------------------------")
#   train_loop(train_dataloader, model, loss_fn, optimizer)
#   test_loop(test_dataloader, model, loss_fn)
# print("Done!")

In [ ]:
# # label 2
# def train_loop(dataloader, model, loss_fn, optimizer):
#   size = len(dataloader.dataset)
#   for batch, (X, y) in enumerate(dataloader):
#   # Compute prediction and loss
#     X = X.cpu()
#     y = list(y)
#     for i in range(len(y)):
#       y[i] = int(y[i])
#       if y[i] == 2:
#         y[i] = 1
#       else:
#         y[i] = 0
#     y = torch.tensor(y)
#     y = y.cpu()
#     pred = model(X)
#     loss = loss_fn(pred, y)
#     # Backpropagation
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if batch % 100 == 0:
#       loss, current = loss.item(), batch * len(X)
#       print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
# def test_loop(dataloader, model, loss_fn):
#   size = len(dataloader.dataset)
#   test_loss, correct = 0, 0
#   with torch.no_grad():
#     for X, y in dataloader:
#       X = X.cpu()
#       y = list(y)
#       for i in range(len(y)):
#         y[i] = int(y[i])
#         if y[i] == 2:
#           y[i] = 1
#         else:
#           y[i] = 0
#       y = torch.tensor(y)
#       y = y.cpu()
#       pred = model(X)
#       test_loss += loss_fn(pred, y).item()
#       correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= size # (TP + TN)/(TP + TN + FP + FN) i think
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# epochs = 7
# for t in range(epochs):
#   print(f"Epoch {t+1}\n-------------------------------")
#   train_loop(train_dataloader, model, loss_fn, optimizer)
#   test_loop(test_dataloader, model, loss_fn)
# print("Done!")

In [ ]:
# # label 4
# def train_loop(dataloader, model, loss_fn, optimizer):
#   size = len(dataloader.dataset)
#   for batch, (X, y) in enumerate(dataloader):
#   # Compute prediction and loss
#     X = X.cpu()
#     y = list(y)
#     for i in range(len(y)):
#       y[i] = int(y[i])
#       if y[i] == 4:
#         y[i] = 1
#       else:
#         y[i] = 0
#     y = torch.tensor(y)
#     y = y.cpu()
#     pred = model(X)
#     loss = loss_fn(pred, y)
#     # Backpropagation
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if batch % 100 == 0:
#       loss, current = loss.item(), batch * len(X)
#       print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
# def test_loop(dataloader, model, loss_fn):
#   size = len(dataloader.dataset)
#   test_loss, correct = 0, 0
#   with torch.no_grad():
#     for X, y in dataloader:
#       X = X.cpu()
#       y = list(y)
#       for i in range(len(y)):
#         y[i] = int(y[i])
#         if y[i] == 4:
#           y[i] = 1
#         else:
#           y[i] = 0
#       y = torch.tensor(y)
#       y = y.cpu()
#       pred = model(X)
#       test_loss += loss_fn(pred, y).item()
#       correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= size # (TP + TN)/(TP + TN + FP + FN) i think
#     correct /= size
#     print(correct)
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# epochs = 7
# for t in range(epochs):
#   print(f"Epoch {t+1}\n-------------------------------")
#   train_loop(train_dataloader, model, loss_fn, optimizer)
#   test_loop(test_dataloader, model, loss_fn)
# print("Done!")

In [ ]:
# old data proccessing code - BAD
# import numpy as np
# import cv2
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# global img_path
# global input_image
# global input_tensor
# global input_batch
# train_dict = dict()
# test_dict = dict()
# val_dict = dict()
# class RetinoDataset(Dataset):
#   def __init__(self):
#     self.train_set = []
#     self.test_set = []
#     self.validation_set = []
#     for i in range(0, 6):
#       path = f"/content/drive/MyDrive/TrainingData_Final/{i}"
#       all_imgs_0 = os.listdir(path)
#       # /content/drive/MyDrive/TrainingData_Final/0
#       all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
#       num_imgs_0 = len(all_imgs_0)
#       arr = np.arange(num_imgs_0)
#       np.random.shuffle(arr)
#       self.train_set += [all_imgs_0[idx] for idx in arr[0:int(0.05*num_imgs_0)]]
#       self.test_set += [all_imgs_0[idx]  for idx in arr[int(0.05*num_imgs_0):int(0.9*num_imgs_0)]]
#       self.validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
#       # print(self.train_set)
#     # self.__len__()
#     # self.main()
#     # self.__getitem()



#     self.train_data = self.train_set
#     self.test_data = self.test_set
#     self.val_data = self.validation_set

#       #transpose
#     self.preprocess = transforms.Compose([
#         transforms.Resize((256, 256)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])

#   # def __len__(self):
#     # return self.train_data
#     # return len(self.test_data)
#     # return len(self.val_data)
#     # return(self.data_sets)
#   # global img_path
#   # global input_image
#   # global input_tensor
#   # global input_batch
#   def __getitem(self, curr_dataset, idx):
#     # print('idx:', idx)
#     # print('len:', len(idx))
#     # print(curr_dataset)
#     img_path = idx[0]
#     input_image = Image.open(img_path)
#     input_tensor = self.preprocess(input_image)
#     input_batch = input_tensor.unsqueeze(0)
#     # print(input_batch)

#     return {"image": input_batch, "label": int(img_path[42])}

#   def main(self):
#     train_dataloader = DataLoader(self.train_data, batch_size=1)
#     test_dataloader = DataLoader(self.test_data, batch_size=1)
#     val_dataloader = DataLoader(self.val_data, batch_size=1)
#     # print(train_dataloader)

#     for img in tqdm(train_dataloader):

#       # print(img["image"].shape)
#       train_dict[] = (self.__getitem(self.train_set, img))
#       # cv2.imwrite(img[0], input_image)
#     print('Complete!')
# # main()
# dataset = RetinoDataset()
# dataset.main()
# print(dataset)

In [ ]:
# labels_map = {
#     0: "Mild Diabetic Nonproliferative Retinopathy",
#     1: "Moderate Diabetic Nonproliferative Retinopathy",
#     2: "Severe Diabetic Nonproliferative Retinopathy",
#     3: "Hypertensive Retinopathy",
#     4: "Proliferative Diabetic Retinopathy",
#     5: "No Retinopathy"
# }

# # figure = plt.figure(figsize=(512, 512))
# # cols, rows = 3, 3
# # train = '/content/Train_Test_Folder/test/FinalTrainingData'
# # test = '/content/Train_Test_Folder/train/FinalTrainingData'
# # print(train.count())
# # print(test.count())
# # training_data = datasets(
# #     root=train,
# #     train=True,
# #     download=True,
# #     transform=ToTensor()
# # )

# # test_data = datasets(
# #     root=test,
# #     train=False,
# #     download=True,
# #     transform=ToTensor()
# # )
# for i in range(1, cols * rows + 1):
#     sample_idx = torch.randint(len(train), size=(1,)).item()
#     img = train[sample_idx]
#     figure.add_subplot(rows, cols, i)
#     plt.title(labels_map[0])
#     plt.axis("off")
#     plt.imshow(img.squeeze(), cmap="gray")
# plt.show()

In [ ]:
# print("Model structure: ", model, "\n\n")
# for name, param in model.named_parameters():
  # print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")


In [ ]:
# for batch, (X, y) in enumerate(dataloader):
#   optimizer.zero_grad()
# # Compute prediction and loss
#   X = X.cpu()
#   y = list(y)
#   for i in range(len(y)):
#     y[i] = int(y[i])
#     if y[i] == 0:
#       y[i] = 1
#     else:
#       y[i] = 0
#   y = torch.tensor(y)
#   y = y.cpu()
#   history = model.fit(X, y, batch_size=8, epochs=50, validation_data=(X_valid, y_valid))
# for t in range(epochs):
#   print(f"Epoch {t+1}\n-------------------------------")
#   train_loop(train_dataloader, model, loss_fn, optimizer)
#   test_loop(test_dataloader, model, loss_fn)
# print("Done!")

In [ ]:
# RYAN COMPUTER CODE
# data = '/content/drive/MyDrive/UCSB SRA Project/TrainingData_Final'
# print()
# dirs = os.listdir(data)
# dataset = []
# for dir in dirs:
#   for inner_dir in os.listdir(os.path.join(data, dir)):
#       dataset.append(os.path.join(data, dir, inner_dir))
# print(dataset)

In [ ]:
# import numpy as np
# train_set = []
# test_set = []
# validation_set = []
# for i in range(0, 6):
#   path = f"/content/drive/MyDrive/TrainingData_Final/{i}"
#   all_imgs_0 = os.listdir(path)
#   # /content/drive/MyDrive/TrainingData_Final/0
#   all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
#   num_imgs_0 = len(all_imgs_0)
#   arr = np.arange(num_imgs_0)
#   np.random.shuffle(arr)
#   train_set += [all_imgs_0[idx] for idx in arr[0:int(0.8*num_imgs_0)]]
#   test_set += [all_imgs_0[idx]  for idx in arr[int(0.8*num_imgs_0):int(0.9*num_imgs_0)]]
#   validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
# print(len(train_set))
# print(len(test_set))
# print(len(validation_set))

In [ ]:
# for element in range(len(train_set)):
#   img = Image.open(train_set[element])
#   convert_tensor = transforms.ToTensor()
#   train_set[element] = convert_tensor(img)
#   print(len(train_set[element]))
# for element in range(len(validation_set)):
#   img = Image.open(validation_set[element])
#   convert_tensor = transforms.ToTensor()
#   validation_set[element] = convert_tensor(img)
# for element in range(len(test_set)):
#   img = Image.open(test_set[element])
#   convert_tensor = transforms.ToTensor()
#   test_set[element] = convert_tensor(img)

In [ ]:
# RYAN COMPUTER CODE
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image

# labels = []
# class CustomDataset(Dataset):
#     def __init__(self, file_paths):
#         self.file_paths = file_paths
#         self.transform = transforms.Compose([
#             transforms.Resize((256, 256)),  # Resize the image to (256, 256)
#             transforms.ToTensor(),          # Convert the PIL image to a PyTorch tensor
#             transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the tensor
#         ])

#     def __len__(self):
#         return len(self.file_paths)

#     def __getitem__(self, idx):
#         file_path = self.file_paths[idx]
#         image = Image.open(file_path)

#         if self.transform:
#             image = self.transform(image)


#         return image, file_path[59]
# train_dataloader = DataLoader(CustomDataset(train_set), batch_size=32, shuffle=True)
# test_dataloader = DataLoader(CustomDataset(test_set), batch_size=32, shuffle=True)
# val_dataloader = DataLoader(CustomDataset(validation_set), batch_size=32, shuffle=True)

In [ ]:
# RYAN COMPUTER CODE
# train_set = []
# test_set = []
# validation_set = []
# for i in range(0, 6):
#   path = f"/content/drive/MyDrive/UCSB SRA Project/TrainingData_Final/{i}"
#   all_imgs_0 = os.listdir(path)
#   # /content/drive/MyDrive/TrainingData_Final/0
#   all_imgs_0 = [f"{path}/{_path}" for _path in all_imgs_0]
#   num_imgs_0 = len(all_imgs_0)
#   arr = np.arange(num_imgs_0)
#   np.random.shuffle(arr)
#   train_set += [all_imgs_0[idx] for idx in arr[0:int(0.05*num_imgs_0)]]
#   test_set += [all_imgs_0[idx]  for idx in arr[int(0.05*num_imgs_0):int(0.9*num_imgs_0)]]
#   validation_set += [all_imgs_0[idx]  for idx in arr[int(0.9*num_imgs_0):int(num_imgs_0)]]
